In [50]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import os
from os import listdir
import soundfile
from IPython.display import Audio
import random
librosa.__version__

'0.8.0'

In [102]:
gan_location = 'samples/ICLRdata/GANAudio2/'
smf_location = 'samples/ICLRdata/RNNAudio2/'
nsynth_location = 'samples/ICLRdata/nsynth_subset_train/'

gan_file = gan_location+'test_spectset2snd_d1.{inst_id}_d0.{pitch_id}_v.0.wav'
rnn_file = smf_location+'inst_{inst_id}_pitch_{pitch_id}.wav'
brass_nsynth_file = nsynth_location+'brass_acoustic_018-0{pitch}-127.wav'
reed_nsynth_file = nsynth_location+'reed_acoustic_027-0{pitch}-127.wav'

In [103]:
# SMF and GAN - Instruments: 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20; Pitches: 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16
# NSynth - Instruments: Brass and Reed; Pitches: 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72

# 11 SMF/GAN instruments - Choose 0,4,10,16,20
# 13 pitches

In [104]:
pitches = [i for i in range(60, 73, 1)]
smf_gan_pitches = [i+4 for i, p in enumerate(pitches)]

instruments = ['brass', 'reed']
smf_gan_instruments = [0,4,10,16,20]

In [136]:
df = pd.DataFrame(columns=['pitch_under_test','instrument_under_test',\
    'test_1_type', 'test_1_ref1', 'test_1_ref1_type', 'test_1_ref2', 'test_1_ref2_type', 'test_1_sample', 'test_1_correct_type',\
            'test_2_type', 'test_2_ref1', 'test_2_ref1_type', 'test_2_ref2', 'test_2_ref2_type', 'test_2_sample', 'test_2_correct_type', \
            'test_3_type', 'test_3_ref1', 'test_3_ref1_type', 'test_3_ref2', 'test_3_ref2_type', 'test_3_sample', 'test_3_correct_type', \
            'test_4_type', 'test_4_ref1', 'test_4_ref1_type', 'test_4_ref2', 'test_4_ref2_type', 'test_4_sample', 'test_4_correct_type'])
for ind, pitch in enumerate(pitches):

    smf_gan_pitch = smf_gan_pitches[ind]

    
    for instrument in smf_gan_instruments:
        
        #2GAN, One SMF
        #--------------
        gan_smf_ref1 = ''
        gan_smf_ref1_type = ''
        gan_smf_ref2 = ''
        gan_smf_ref2_type = ''

        if random.random() > 0.5: #brass vs. reed
            gan_smf_ref1 = gan_file.format(inst_id=0, pitch_id=smf_gan_pitch)
            gan_smf_ref1_type = 'test_spectset2snd_d1.0_d0'
            gan_smf_ref2 = gan_file.format(inst_id=20, pitch_id=smf_gan_pitch)
            gan_smf_ref2_type = 'test_spectset2snd_d1.20_d0'
        else: #reed vs. brass
            gan_smf_ref1 = gan_file.format(inst_id=20, pitch_id=smf_gan_pitch)
            gan_smf_ref1_type = 'test_spectset2snd_d1.20_d0'
            gan_smf_ref2 = gan_file.format(inst_id=0, pitch_id=smf_gan_pitch)
            gan_smf_ref2_type = 'test_spectset2snd_d1.0_d0'

        gan_smf_test_sample = rnn_file.format(inst_id=instrument, pitch_id=smf_gan_pitch)

        
        #2Nsynth, One GAN
        #-----------------
        nsynth_gan_ref1 = ''
        nsynth_gan_ref1_type = ''
        nsynth_gan_ref2 = ''
        nsynth_gan_ref2_type = ''

        if random.random() > 0.5: #brass vs. reed
            nsynth_gan_ref1 = brass_nsynth_file.format(pitch=pitch)
            nsynth_gan_ref1_type = 'brass'
            nsynth_gan_ref2 = reed_nsynth_file.format(pitch=pitch)
            nsynth_gan_ref2_type = 'reed'
        else: #reed vs. brass
            nsynth_gan_ref1 = reed_nsynth_file.format(pitch=pitch)
            nsynth_gan_ref1_type = 'reed'
            nsynth_gan_ref2 = brass_nsynth_file.format(pitch=pitch)
            nsynth_gan_ref2_type = 'brass'

        nsynth_gan_test_sample = gan_file.format(inst_id=instrument, pitch_id=smf_gan_pitch)

       
        #2Nsynth, One SMF
        #-----------------
        nsynth_smf_ref1 = ''
        nsynth_smf_ref1_type = ''
        nsynth_smf_ref2 = ''
        nsynth_smf_ref2_type = ''

        if random.random() > 0.5: #brass vs. reed
            nsynth_smf_ref1 = brass_nsynth_file.format(pitch=pitch)
            nsynth_smf_ref1_type = 'brass'
            nsynth_smf_ref2 = reed_nsynth_file.format(pitch=pitch)
            nsynth_smf_ref2_type = 'reed'
        else: #reed vs. brass
            nsynth_smf_ref1 = reed_nsynth_file.format(pitch=pitch)
            nsynth_smf_ref1_type = 'reed'
            nsynth_smf_ref2 = brass_nsynth_file.format(pitch=pitch)
            nsynth_smf_ref2_type = 'brass'

        nsynth_smf_test_sample = rnn_file.format(inst_id=instrument, pitch_id=smf_gan_pitch)


        #Sentinel
        #-----------------
        nsynth_sentinel_ref1 = ''
        nsynth_sentinel_ref1_type = ''
        nsynth_sentinel_ref2 = ''
        nsynth_sentinel_ref2_type = ''
        nsynth_sentinel_correct_type = ''

        if random.random() > 0.5: #brass vs. reed
            nsynth_sentinel_ref1 = brass_nsynth_file.format(pitch=pitch)
            nsynth_sentinel_ref1_type = 'brass'
            nsynth_sentinel_ref2 = reed_nsynth_file.format(pitch=pitch)
            nsynth_sentinel_ref2_type = 'reed'

        else: #reed vs. brass
            nsynth_sentinel_ref1 = reed_nsynth_file.format(pitch=pitch)
            nsynth_sentinel_ref1_type = 'reed'
            nsynth_sentinel_ref2 = brass_nsynth_file.format(pitch=pitch)
            nsynth_sentinel_ref2_type = 'brass'

        if random.random() > 0.5: #brass
            nsynth_sentinel_correct_type = 'brass'
            nsynth_sentinel_test_sample = brass_nsynth_file.format(pitch=pitch)
        else:
            nsynth_sentinel_correct_type = 'reed'
            nsynth_sentinel_test_sample = reed_nsynth_file.format(pitch=pitch)

        
        

        test_types = ['gan_smf', 'nsynth_gan', 'nsynth_smf', 'sentinel']
        random.shuffle(test_types)

        for ind, test_type in enumerate(test_types):
            if test_type == 'gan_smf':
                ref1 = gan_smf_ref1
                ref1_type = gan_smf_ref1_type
                ref2 = gan_smf_ref2
                ref2_type = gan_smf_ref2_type
                test_sample = gan_smf_test_sample
                test_correct_type = ''

            if test_type == 'nsynth_gan':
                ref1 = nsynth_gan_ref1
                ref1_type = nsynth_gan_ref1_type
                ref2 = nsynth_gan_ref2
                ref2_type = nsynth_gan_ref2_type
                test_sample = nsynth_gan_test_sample
                test_correct_type = ''

            if test_type == 'nsynth_smf':
                ref1 = nsynth_smf_ref1
                ref1_type = nsynth_smf_ref1_type
                ref2 = nsynth_smf_ref2
                ref2_type = nsynth_smf_ref2_type
                test_sample = nsynth_smf_test_sample
                test_correct_type = ''

            if test_type == 'sentinel':
                ref1 = nsynth_sentinel_ref1
                ref1_type = nsynth_sentinel_ref1_type
                ref2 = nsynth_sentinel_ref2
                test_sample = nsynth_sentinel_test_sample
                test_correct_type = nsynth_sentinel_correct_type


            if ind == 0:
                test_1_type = test_type
                test_1_ref1 = ref1
                test_1_ref1_type = ref1_type
                test_1_ref2 = ref2
                test_1_ref2_type = ref2_type
                test_1_sample = test_sample
                test_1_correct_type = test_correct_type

            if ind == 1:
                test_2_type = test_type
                test_2_ref1 = ref1
                test_2_ref1_type = ref1_type
                test_2_ref2 = ref2
                test_2_ref2_type = ref2_type
                test_2_sample = test_sample
                test_2_correct_type = test_correct_type
                
            if ind == 2:
                test_3_type = test_type
                test_3_ref1 = ref1
                test_3_ref1_type = ref1_type
                test_3_ref2 = ref2
                test_3_ref2_type = ref2_type
                test_3_sample = test_sample
                test_3_correct_type = test_correct_type

            if ind == 3:
                test_4_type = test_type
                test_4_ref1 = ref1
                test_4_ref1_type = ref1_type
                test_4_ref2 = ref2
                test_4_ref2_type = ref2_type
                test_4_sample = test_sample
                test_4_correct_type = test_correct_type
        
        df.loc[str(pitch)+'_'+str(instrument)] = [pitch, instrument, \
            test_1_type, test_1_ref1, test_1_ref1_type, test_1_ref2, test_1_ref2_type, test_1_sample, test_1_correct_type,\
            test_2_type, test_2_ref1, test_2_ref1_type, test_2_ref2, test_2_ref2_type, test_2_sample, test_2_correct_type, \
            test_3_type, test_3_ref1, test_3_ref1_type, test_3_ref2, test_3_ref2_type, test_3_sample, test_3_correct_type, \
            test_4_type, test_4_ref1, test_4_ref1_type, test_4_ref2, test_4_ref2_type, test_4_sample, test_4_correct_type]

# df = df.sample(frac=1).reset_index(drop=True)
df.to_csv('test.csv')

In [137]:
df

,pitch_under_test,instrument_under_test,test_1_type,test_1_ref1,test_1_ref1_type,test_1_ref2,test_1_ref2_type,test_1_sample,test_1_correct_type,test_2_type,...,test_3_ref2_type,test_3_sample,test_3_correct_type,test_4_type,test_4_ref1,test_4_ref1_type,test_4_ref2,test_4_ref2_type,test_4_sample,test_4_correct_type
60_0,60,0,gan_smf,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,test_spectset2snd_d1.0_d0,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,test_spectset2snd_d1.20_d0,samples/ICLRdata/RNNAudio2/inst_0_pitch_4.wav,,sentinel,...,brass,samples/ICLRdata/RNNAudio2/inst_0_pitch_4.wav,,nsynth_gan,samples/ICLRdata/nsynth_subset_train/brass_aco...,brass,samples/ICLRdata/nsynth_subset_train/reed_acou...,reed,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,
60_4,60,4,nsynth_smf,samples/ICLRdata/nsynth_subset_train/brass_aco...,brass,samples/ICLRdata/nsynth_subset_train/reed_acou...,reed,samples/ICLRdata/RNNAudio2/inst_4_pitch_4.wav,,gan_smf,...,reed,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,,sentinel,samples/ICLRdata/nsynth_subset_train/brass_aco...,brass,samples/ICLRdata/nsynth_subset_train/reed_acou...,reed,samples/ICLRdata/nsynth_subset_train/reed_acou...,reed
60_10,60,10,sentinel,samples/ICLRdata/nsynth_subset_train/reed_acou...,reed,samples/ICLRdata/nsynth_subset_train/brass_aco...,reed,samples/ICLRdata/nsynth_subset_train/reed_acou...,reed,gan_smf,...,brass,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,,nsynth_smf,samples/ICLRdata/nsynth_subset_train/brass_aco...,brass,samples/ICLRdata/nsynth_subset_train/reed_acou...,reed,samples/ICLRdata/RNNAudio2/inst_10_pitch_4.wav,
60_16,60,16,nsynth_smf,samples/ICLRdata/nsynth_subset_train/brass_aco...,brass,samples/ICLRdata/nsynth_subset_train/reed_acou...,reed,samples/ICLRdata/RNNAudio2/inst_16_pitch_4.wav,,sentinel,...,brass,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,,gan_smf,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,test_spectset2snd_d1.0_d0,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,test_spectset2snd_d1.20_d0,samples/ICLRdata/RNNAudio2/inst_16_pitch_4.wav,
60_20,60,20,sentinel,samples/ICLRdata/nsynth_subset_train/reed_acou...,reed,samples/ICLRdata/nsynth_subset_train/brass_aco...,test_spectset2snd_d1.20_d0,samples/ICLRdata/nsynth_subset_train/brass_aco...,brass,nsynth_smf,...,reed,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,,gan_smf,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,test_spectset2snd_d1.0_d0,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,test_spectset2snd_d1.20_d0,samples/ICLRdata/RNNAudio2/inst_20_pitch_4.wav,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72_0,72,0,gan_smf,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,test_spectset2snd_d1.0_d0,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,test_spectset2snd_d1.20_d0,samples/ICLRdata/RNNAudio2/inst_0_pitch_16.wav,,sentinel,...,reed,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,,nsynth_smf,samples/ICLRdata/nsynth_subset_train/brass_aco...,brass,samples/ICLRdata/nsynth_subset_train/reed_acou...,reed,samples/ICLRdata/RNNAudio2/inst_0_pitch_16.wav,
72_4,72,4,gan_smf,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,test_spectset2snd_d1.20_d0,samples/ICLRdata/GANAudio2/test_spectset2snd_d...,test_spectset2snd_d1.0_d0,samples/ICLRdata/RNNAudio2/inst_4_pitch_16.wav,,nsynth_gan,...,brass,samples/ICLRdata/RNNAudio2/inst_4_pitch_16.wav,,sentinel,samples/ICLRdata/nsynth_subset_train/reed_acou...,reed,samples/ICLRdata/nsynth_subset_train/brass_aco...,brass,samples/ICLRdata/nsynth_subset_train/brass_aco...,brass
72_10,72,10,sentinel,samples/ICLRdata/nsynth_subset_train/reed_acou...,reed,samples/ICLRdata/nsynth_subset_train/brass_aco...,brass,samples/ICLRdata/nsynth_subset_train/brass_aco...,brass,nsynth_gan,...,test_spectset2snd_d1.20_d0,samples/ICLRdata/RNNAudio2/inst_10_pitch_16.wav,,nsynth_smf,samples/ICLRdata/nsynth_subset_train/brass_aco...,brass,samples/ICLRdata/nsynth_subset_train/reed_acou...,reed,samples/ICLRdata/RNN

In [138]:
def get_test_name(type):
    if type == 'nsynth_smf':
        return 'Reference: NSynth; Test: SMF'
    if type == 'nsynth_gan':
        return 'Reference: NSynth; Test: GAN'
    if type == 'gan_smf':
        return 'Reference: GAN; Test: SMF'
    if type == 'sentinel':
        return 'Sentinel'

In [139]:
s = '<table><tr>\
         <td style=''><h3>\
         Pitch = {pitch} and Instrument = {instrument}\
         </h3></td>\
    </tr>\
    <tr>\
         <td>\
         <b>Test 1: {test_1_type} - </b>\
         </td>\
        <td>\
            Reference 1: \
        </td>\
        <td>\
            <audio controls="">\
                <source src="{test_1_ref1}" type="audio/wav" />\
            </audio>\
        </td>\
        <td>\
            Reference 2: \
        </td>\
        <td>\
            <audio controls="">\
                <source src="{test_1_ref2}" type="audio/wav" />\
            </audio>\
        </td>\
        <td>\
            Sample under test: \
        </td>\
        <td>\
            <audio controls="">\
                <source src="{test_1_sample}" type="audio/wav" />\
            </audio>\
        </td>\
    </tr>\
        <hr/>\
    <tr>\
        <td>&nbsp;</td>\
    </tr>\
    <tr>\
         <td style=''>\
         <b>Test 2: {test_2_type} - </b>\
         </td>\
        <td>\
            Reference 1: \
        </td>\
        <td>\
            <audio controls="">\
                <source src="{test_2_ref1}" type="audio/wav" />\
            </audio>\
        </td>\
        <td>\
            Reference 2: \
        </td>\
        <td>\
            <audio controls="">\
                <source src="{test_2_ref2}" type="audio/wav" />\
            </audio>\
        </td>\
        <td>\
            Sample under test: \
        </td>\
        <td>\
            <audio controls="">\
                <source src="{test_2_sample}" type="audio/wav" />\
            </audio>\
        </td>\
    </tr>\
    <tr>\
        <td>&nbsp;</td>\
    </tr>\
    <tr>\
         <td style=''>\
         <b>Test 3: {test_3_type} - </b>\
         </td>\
        <td>\
            Reference 1: \
        </td>\
        <td>\
            <audio controls="">\
                <source src="{test_3_ref1}" type="audio/wav" />\
            </audio>\
        </td>\
        <td>\
            Reference 2: \
        </td>\
        <td>\
            <audio controls="">\
                <source src="{test_3_ref2}" type="audio/wav" />\
            </audio>\
        </td>\
        <td>\
            Sample under test: \
        </td>\
        <td>\
            <audio controls="">\
                <source src="{test_3_sample}" type="audio/wav" />\
            </audio>\
        </td>\
    </tr>\
    <tr>\
        <td>&nbsp;</td>\
    </tr>\
    <tr>\
         <td style=''>\
         <b>Test 4: {test_4_type} - </b>\
         </td>\
        <td>\
            Reference 1: \
        </td>\
        <td>\
            <audio controls="">\
                <source src="{test_4_ref1}" type="audio/wav" />\
            </audio>\
        </td>\
        <td>\
            Reference 2: \
        </td>\
        <td>\
            <audio controls="">\
                <source src="{test_4_ref2}" type="audio/wav" />\
            </audio>\
        </td>\
        <td>\
            Sample under test: \
        </td>\
        <td>\
            <audio controls="">\
                <source src="{test_4_sample}" type="audio/wav" />\
            </audio>\
        </td>\
    </tr>\
    </table>'


In [143]:
count = 0
for ind, row in df.iterrows():
    # if count <= 20:
    print(s.format(pitch=row['pitch_under_test'], instrument=row['instrument_under_test'],\
    test_1_type=get_test_name(row['test_1_type']),test_1_ref1=row['test_1_ref1'],test_1_ref2=row['test_1_ref2'],test_1_sample=row['test_1_sample'],\
    test_2_type=get_test_name(row['test_2_type']),test_2_ref1=row['test_2_ref1'],test_2_ref2=row['test_2_ref2'],test_2_sample=row['test_2_sample'],\
    test_3_type=get_test_name(row['test_3_type']),test_3_ref1=row['test_3_ref1'],test_3_ref2=row['test_3_ref2'],test_3_sample=row['test_3_sample'],\
    test_4_type=get_test_name(row['test_4_type']),test_4_ref1=row['test_4_ref1'],test_4_ref2=row['test_4_ref2'],test_4_sample=row['test_4_sample']))
    count += 1

<table><tr>         <td style=><h3>         Pitch = 60 and Instrument = 0         </h3></td>    </tr>    <tr>         <td>         <b>Test 1: Reference: GAN; Test: SMF - </b>         </td>        <td>            Reference 1:         </td>        <td>            <audio controls="">                <source src="samples/ICLRdata/GANAudio2/test_spectset2snd_d1.0_d0.4_v.0.wav" type="audio/wav" />            </audio>        </td>        <td>            Reference 2:         </td>        <td>            <audio controls="">                <source src="samples/ICLRdata/GANAudio2/test_spectset2snd_d1.20_d0.4_v.0.wav" type="audio/wav" />            </audio>        </td>        <td>            Sample under test:         </td>        <td>            <audio controls="">                <source src="samples/ICLRdata/RNNAudio2/inst_0_pitch_4.wav" type="audio/wav" />            </audio>        </td>    </tr>        <hr/>    <tr>        <td>&nbsp;</td>    </tr>    <tr>         <td style=>         <b>Test 2